In [24]:
import json
import requests
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import matplotlib.dates as mdates
from geopy.adapters import AioHTTPAdapter
from geopy.geocoders import Nominatim


In [12]:
mcdw = pd.read_csv('../data/mcdw.20230530.csv')
mcdw.info()

C:\Users\Aaron\AppData\Local\Temp\ipykernel_38564\2137799343.py:1: DtypeWarning: Columns (9,13,19) have mixed types. Specify dtype option on import or set low_memory=False.
  mcdw = pd.read_csv('../data/mcdw.20230530.csv')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 907855 entries, 0 to 907854
Data columns (total 36 columns):
 #   Column                                             Non-Null Count   Dtype  
---  ------                                             --------------   -----  
 0   WMO ID                                             907855 non-null  int64  
 1   YEAR-MONTH                                         907855 non-null  object 
 2   LONGITUDE (deg)                                    907855 non-null  float64
 3   LATITUDE (deg)                                     907855 non-null  float64
 4   ELEVATION (m)                                      907855 non-null  float64
 5   STATION PRESSURE (0.1 mb)                          907855 non-null  int64  
 6   STATION PRESSURE SOURCE FLAG                       826655 non-null  object 
 7   MEAN SEA LEVEL PRESSURE (0.1 mb)                   825674 non-null  float64
 8   MEAN SEA LEVEL PRESSURE SOURCE FLAG                907855 non-null  object

In [14]:
#Droppping columns not relavent 
mcdw.drop(['ELEVATION (m)', 'STATION PRESSURE (0.1 mb)','STATION PRESSURE SOURCE FLAG', 'MEAN SEA LEVEL PRESSURE (0.1 mb)', 'MEAN SEA LEVEL PRESSURE SOURCE FLAG', 'MEAN SEA LEVEL PRESSURE HEIGHT FLAG', 'VAPOR PRESSURE (0.1 mb)', 'VAPOR PRESSURE SOURCE FLAG', 'VAPOR PRESSURE DEPARTURE (0.1 mb)', 'VAPOR PRESSURE DEPARTURE SOURCE FLAG'], axis=1, inplace=True)

In [26]:
#removing data with unknow coordinates
mcdw = mcdw[(mcdw['LATITUDE (deg)'] != -9999.0000) & (mcdw['LONGITUDE (deg)'] != -9999.0000)]

In [28]:
#mcdw['lat_long'] = mcdw['LATITUDE (deg)'] & mcdw['LONGITUDE (deg)']
mcdw['lat_long']=mcdw.apply(lambda x:'%s,%s' % (x['LATITUDE (deg)'],x['LONGITUDE (deg)']),axis=1)

C:\Users\Aaron\AppData\Local\Temp\ipykernel_38564\1519233826.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mcdw['lat_long']=mcdw.apply(lambda x:'%s,%s' % (x['LATITUDE (deg)'],x['LONGITUDE (deg)']),axis=1)


In [29]:
mcdw

,WMO ID,YEAR-MONTH,LONGITUDE (deg),LATITUDE (deg),MINIMUM TEMPERATURE (deg C),MINIMUM TEMPERATURE SOURCE FLAG,MEAN TEMPERATURE (deg C),MEAN TEMPERATURE SOURCE FLAG,MEAN TEMPERATURE DEPARTURE (deg C),MEAN TEMPERATURE DEPARTURE SOURCE FLAG,...,TOTAL PRECIPITATION DEPARTURE SOURCE FLAG,PRECIPITATION QUINTILE,PRECIPITATION QUINTILE SOURCE FLAG,SUNSHINE DURATION (hours),SUNSHINE DURATION SOURCE FLAG,SUNSHINE DURATION AS LONG TERM NORMAL (%),SUNSHINE DURATION AS LONG TERM NORMAL SOURCE FLAG,VERSION,STATION NAME,lat_long
1,1001,1987-01,-8.6678,70.9397,-9999,NaN,-26,L,31,L,...,L,3,L,-9999,NaN,-9999,NaN,mcdw.20230530,JAN MAYEN (1001-1),"70.9397,-8.6678"
2,1001,1987-02,-8.6678,70.9397,-9999,NaN,-93,L,-32,L,...,L,3,L,-9999,NaN,-9999,NaN,mcdw.20230530,JAN MAYEN (1001-1),"70.9397,-8.6678"
3,1001,1987-03,-8.6678,70.9397,-9999,NaN,-42,L,19,L,...,L,5,L,-9999,NaN,-9999,NaN,mcdw.20230530,JAN MAYEN (1001-1),"70.9397,-8.6678"
4,1001,1987-04,-8.6678,70.9397,-9999,NaN,-16,L,23,L,...,L,6,L,-9999,NaN,-9999,NaN,mcdw.20230530,JAN MAYEN (1001-1),"70.9397,-8.6678"
5,1001,1987-05,-8.6678,70.9397,-9999,NaN,2,L,9,L,...,L,4,L,-9999,NaN,-9999,NaN,mcdw.20230530,JAN MAYEN (1001-1),"70.9397,-8.6678"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
907770,98851,2023-03,125.1030,6.0572,230,B,281,B,-9999,NaN,...,NaN,3,B,-9999,NaN,-9999,NaN,mcdw.20230530,GENERAL SANTOS,"6.0572,125.103"
907771,98851,2023-04,125.1030,6.0572,232,B,281,B,-9999,NaN,...,NaN,3,B,-9999,NaN,-9999,NaN,mcdw.20230530,GENERAL SANTOS,"6.0572,125.103"
907852,99999,2020-09,8.9783,46.3361,0,B,0,B,-9999,NaN,...,NaN,0,B,0,B,0,B,mcdw.20230530,BIASCA,"46.3361,8.9783"
907853,99999,2021-04,8.9783,46.3361,0,B,0,B,-9999,NaN,...,NaN,0,B,0,B,0,B,mcdw.20230530,BIASCA,"46.3361,8.9783"


In [35]:
#Need to group into regions
#location = geolocator.reverse("{}, {}".format(mcdw['LATITUDE (deg)'], mcdw['LONGITUDE (deg)']))

geolocator = Nominatim(user_agent="gc123")
#location = geolocator.reverse(mcdw['lat_long'])
 
#mcdw["location"] = mcdw['lat_long'].apply(lambda el: geolocator.reverse(el).raw["address"])
mcdw["city"] = list(map(lambda el: geolocator.reverse(el).raw["address"], mcdw["lat_long"]))

KeyboardInterrupt: 